In [1]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import imdb

Using Theano backend.


In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [3]:
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [4]:
from itertools import chain
max_features = len(set(chain.from_iterable(X_train)) and set(chain.from_iterable(X_test)))

In [9]:
from keras.layers import Convolution1D
from keras.layers.convolutional import Conv1D
num_filters = 150
kernel_size = 3
filter_length = 3

むしろ
http://qiita.com/HirofumiYashima/items/2aee883b736ee600899b
を参考にCNNを構成しよう

In [10]:
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))

filter : アウトプット次元（＝フィルターの数）
kernel_size :ウィンドウのサイズ

In [11]:
model.add(Convolution1D(nb_filter = num_filters , filter_length = filter_length))
#model.add(Conv1D(filter = num_filters , kernel_size = kernel_size))

In [ ]:
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun

In [ ]:
model.add(Dense(1))
model.add(Activation('sigmoid'))